In [1]:
import cv2
import numpy as np
import os

def detect_green_areas(input_dir, output_dir):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    def increase_saturation(image, boost_factor=2):
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv[:, :, 1] = np.clip(hsv[:, :, 1] * boost_factor, 0, 255)
        return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # Loop through every file in the input directory
    for filename in os.listdir(input_dir):

        if filename.endswith(('.jpg', '.jpeg', '.png','.tif')):  # You can add more file extensions if needed
            image_path = os.path.join(input_dir, filename)
            
            # Read the image
            image = cv2.imread(image_path)

            increase_saturation(image)

            # Convert image to HSV
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # Further expanded range for green color
            lower_green = np.array([30, 6, 25])
            upper_green = np.array([140, 255, 255])


            # Create a binary mask where green regions are white
            mask = cv2.inRange(hsv, lower_green, upper_green)

            
            # Morphological operations to remove noise
            kernel = np.ones((5, 5), np.uint8)
            #mask = cv2.erode(mask, kernel, iterations=1)
            mask = cv2.dilate(mask, kernel, iterations=1)

            

            # Create a white background image
            white_background = np.ones_like(image) * 255

            # Merge the original image and the white background using the mask
            result = cv2.bitwise_and(image, image, mask=mask)
            result += cv2.bitwise_and(white_background, white_background, mask=cv2.bitwise_not(mask))

            # Save the result to the output directory
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, result)

# Paths to the directories
input_directory = r"D:\Projects\Weed Detection\data\grass\dataset\grass"
output_directory = r"D:\Projects\Weed Detection\data\grass\dataset\grass_processed"
detect_green_areas(input_directory, output_directory)